In [1]:
import tensorflow as tf

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras import optimizers
from pandas import Series
import math
import numpy
from tensorflow.keras.layers import *
from tensorflow.keras import layers
import pandas as pd



In [2]:

# basic
import pandas as pd
import numpy as np

# cross validation
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold

# model
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

#plot
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred)))

In [4]:
data = pd.read_csv("SeoulBikeData (1).csv",encoding='CP949')

In [5]:
data

,Date,Rented Bike Count,Hour,Temperature,Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature,Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,30/11/2018,1003,19,4.2,34,2.6,1894,-10.3,0.0,0.0,0.0,Autumn,No Holiday,Yes
8756,30/11/2018,764,20,3.4,37,2.3,2000,-9.9,0.0,0.0,0.0,Autumn,No Holiday,Yes
8757,30/11/2018,694,21,2.6,39,0.3,1968,-9.9,0.0,0.0,0.0,Autumn,No Holiday,Yes
8758,30/11/2018,712,22,2.1,41,1.0,1859,-9.8,0.0,0.0,0.0,Autumn,No Holiday,Yes


In [13]:
cols=data.columns[1:-3]
data[cols]

,Rented Bike Count,Hour,Temperature,Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature,Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm)
0,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0
1,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0
2,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0
3,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0
4,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
8755,1003,19,4.2,34,2.6,1894,-10.3,0.0,0.0,0.0
8756,764,20,3.4,37,2.3,2000,-9.9,0.0,0.0,0.0
8757,694,21,2.6,39,0.3,1968,-9.9,0.0,0.0,0.0
8758,712,22,2.1,41,1.0,1859,-9.8,0.0,0.0,0.0


In [8]:
values = data[cols].values.reshape(-1,9)


In [14]:
def fill_missing(data):
    one_day = 7 * 24
    for row in range(data.shape[0]):
        for col in range(data.shape[1]):
            if data[row,0]==0:
                data[row,0] = data[row - one_day,0]

In [15]:
fill_missing(data)


KeyError: (0, 0)

In [11]:
values

array([[254. ,  -5.2,  37. , ...,   0. ,   0. ,   0. ],
       [204. ,  -5.5,  38. , ...,   0. ,   0. ,   0. ],
       [173. ,  -6. ,  39. , ...,   0. ,   0. ,   0. ],
       ...,
       [694. ,   2.6,  39. , ...,   0. ,   0. ,   0. ],
       [712. ,   2.1,  41. , ...,   0. ,   0. ,   0. ],
       [584. ,   1.9,  43. , ...,   0. ,   0. ,   0. ]])

In [9]:
from sklearn.preprocessing import MinMaxScaler
values = data.values.reshape(-1,6)
values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
#scaled = (values-(np.min(values)))/(np.max(values)-(np.min(values)))

In [10]:
train_size = int(len(values) * 0.80)
val_size = int(len(values) * 0.10)

test_size = len(values) - train_size-val_size
train,val, test,test1 = values[0:train_size,:],values[train_size:train_size+val_size,:], values[train_size+val_size:,:],values[train_size+val_size:,:]
print(len(train),len(val), len(test),len(test1))

8004 1000 1001 1001


In [11]:
def create_dataset_mul(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    print(len(dataY))
    return np.array(dataX), np.array(dataY)

In [11]:
look_back = 24
trainX, trainY = create_dataset_mul(train, look_back)
valX, valY = create_dataset_mul(val, look_back)
testX, testY = create_dataset_mul(test, look_back)
test1X, test1Y = create_dataset_mul(test1, look_back)

7980
976
977
977


In [15]:
train_y.shape

(7980,)

In [28]:
np.array(data.close).shape

(10005,)

In [19]:
np.array(trainY)

array([ 7227.46,  7235.63,  7236.75, ..., 15372.5 , 15378.9 , 15387.48],
      dtype=float32)

In [24]:
np.array(trainY.tolist())

array([ 7227.45996094,  7235.62988281,  7236.75      , ...,
       15372.5       , 15378.90039062, 15387.48046875])

In [29]:
train_y.shape

(7980,)

In [17]:
from sklearn.model_selection import TimeSeriesSplit
r=[]
arr=np.array([])
d=[9004,9011,9018]

for i in range(0,len(d)-1):
    Train=data.iloc[:9004,:]
    Test=data.iloc[9004:,:]
    train_x=Train.drop('close', axis=1)
    train_y=Train[['close']]
    test_x=Test.drop('close',axis=1)
    train_x=np.array(train_x)
    train_y=np.array(train_y)
    test_x=np.array(test_x)
    
    #print(train_y)
    
    folds=[]
    tscv = TimeSeriesSplit(n_splits=3)

    for train_idx, valid_idx in tscv.split(train_x,train_y):
        folds.append((train_idx, valid_idx))
        
    models={}        
    for fold in range(3):
        print(f'===================={fold+1}=======================')
        train_idx, valid_idx=folds[fold]
        X_train=train_x[train_idx, :]
        y_train=train_y[train_idx, :]
        X_valid=train_x[valid_idx, :]
        y_valid=train_y[valid_idx, :]
        print(X_train.shape)
        model=LGBMRegressor(n_estimators=500)
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], 
                 early_stopping_rounds=30, verbose=100)
        models[fold]=model
    
        print(f'================================================\n\n')
        
    answer=0
    for j in range(3):
        answer += models[j].predict(np.array(test_x))/3

====================1=======================
(2251, 5)
Training until validation scores don't improve for 30 rounds


C:\Users\chlgy\Anaconda3\envs\tf2\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\chlgy\Anaconda3\envs\tf2\lib\site-packages\lightgbm\basic.py:154: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[100]	training's l2: 878.092	valid_1's l2: 5385.59
Early stopping, best iteration is:
[78]	training's l2: 944.119	valid_1's l2: 5322.53


====================2=======================
(4502, 5)
Training until validation scores don't improve for 30 rounds


C:\Users\chlgy\Anaconda3\envs\tf2\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\chlgy\Anaconda3\envs\tf2\lib\site-packages\lightgbm\basic.py:154: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[100]	training's l2: 713.541	valid_1's l2: 631609
[200]	training's l2: 565.861	valid_1's l2: 628674
Early stopping, best iteration is:
[229]	training's l2: 532.791	valid_1's l2: 628410


====================3=======================
(6753, 5)
Training until validation scores don't improve for 30 rounds
[100]	training's l2: 693.512	valid_1's l2: 2.02049e+07


C:\Users\chlgy\Anaconda3\envs\tf2\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\chlgy\Anaconda3\envs\tf2\lib\site-packages\lightgbm\basic.py:154: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[200]	training's l2: 569.051	valid_1's l2: 2.01865e+07
Early stopping, best iteration is:
[264]	training's l2: 515.073	valid_1's l2: 2.0173e+07




In [18]:
answer

array([11036.86492115, 11030.6798014 , 11030.38600033, ...,
       11026.48905731, 11017.98914012, 11017.50779587])

In [19]:
MAPE(Test.close,answer)


21288.288761560063

In [30]:
Test.close

8956     23304.15
8957     23553.28
8958     23983.35
8959     23908.02
8960     23845.66
           ...   
10000    33675.45
10001    33590.20
10002    33576.82
10003    33803.02
10004    33897.06
Name: close, Length: 1049, dtype: float64